In [1]:
!pip install pandas
!pip install numpy

In [2]:
import pandas as pd
import numpy as np
import warnings;warnings.filterwarnings('ignore')

In [9]:
movies = pd.read_csv('tmdb_5000_movies.csv')

In [10]:
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [23]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]
print(movies_df)

id                                     title  \
0      19995                                    Avatar   
1        285  Pirates of the Caribbean: At World's End   
2     206647                                   Spectre   
3      49026                     The Dark Knight Rises   
4      49529                               John Carter   
...      ...                                       ...   
4798    9367                               El Mariachi   
4799   72766                                 Newlyweds   
4800  231617                 Signed, Sealed, Delivered   
4801  126186                          Shanghai Calling   
4802   25975                         My Date with Drew   

                                                                                                   genres  \
0     [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {...   
1          [{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": 

In [30]:
pd.set_option('max_colwidth',100)
#각각의 장르와 그 장르의 id를 추출해 낼 수 있다
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{'name': 'Action', 'id': 28}, {'name': 'Adventure', 'id': 12}, {'name': 'Fantasy', 'id': 14}, {...","[{'name': 'culture clash', 'id': 1463}, {'name': 'future', 'id': 2964}, {'name': 'space war', 'i..."


In [31]:
#literal_eval() 함수를 사용하면 이 문자열을 문자열이 의미하는 list[dict1, dict2] 객체로 만들 수 있다
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval) #genres 칼럼은 문자열이 아닌 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체들을 가진다
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)


ValueError: malformed node or string: [{'name': 'Action', 'id': 28}, {'name': 'Adventure', 'id': 12}, {'name': 'Fantasy', 'id': 14}, {'name': 'Science Fiction', 'id': 878}]

In [32]:
#genres 칼럼에서 'name' 키에 해당하는 값을 추출하기 위해서 apply lambda 식을 이용한다
#apply(lambda x : [y['name] for y in x])와 같이 변환하면 리스트 내 여러개의 딕셔너리의 'name'에 해당하는 값을 찾아 이를 리스트 객체로 변환한다
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres','keywords'][:1]]


,genres
0,"[Action, Adventure, Fantasy, Science Fiction]"
1,"[Adventure, Fantasy, Action]"
2,"[Action, Adventure, Crime]"
3,"[Action, Crime, Drama, Thriller]"
4,"[Action, Adventure, Science Fiction]"
...,...
4798,"[Action, Crime, Thriller]"
4799,"[Comedy, Romance]"
4800,"[Comedy, Drama, Romance, TV Movie]"
4801,[]


In [ ]:
#genres 칼럼은 여러개의 개별장르가 리스트로 구성되어 있다
#만일 영화 A가 genres의 [Action, Adventure,Fantasy, Science Fiction]으로 되어 있고
#영화 B의 genres가 [Adventure, Fantasy, Action]으로 돼 있다면 어떻게 장르별 유사도를 측정할 수 있을까?

#가장 간단한 방법으로는 genres를 문자열로 변경한 뒤 이를 CountVectorizer로 피처 벡터화한 행렬 데이터 값을 코사인 유사도로 비교하는 것

#genres 칼럼을 기반으로 하는 콘텐츠 기반 필터링
#1. 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환합니다
#2. genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교합니다. 이를 위해 데이터 세트의 레코드별로 다 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성합니다
#3. 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천한다
